In [1]:
# @title
### EP3 MAC0417 / MAC5768 ##################################################
#  AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,              #
#  DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELAS RESPOSTAS NESTA LISTA.    #
#  TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, SEGUINDO AS    #
#  INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM DESONESTIDADE ACADÊMICA OU PLÁGIO.#
#                                                                              #
#  DECLARO TAMBÉM QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS                      #
#  DESSE PROGRAMA E QUE EU NÃO DISTRIBUI OU FACILITEI A                        #
#  SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E                     #
#  DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS                 #
#  DEFINIDOS NO CÓDIGO DE ÉTICA DA USP.                                        #
#  ENTENDO QUE LISTAS SEM ASSINATURA NÃO SERÃO CORRIGIDAS E,                   #
#  AINDA ASSIM, PODERÃO SER PUNIDOS POR DESONESTIDADE ACADÊMICA.               #
#                                                                              #
#  Nome : Érico Tiago Meireles                                                 #
#  NUSP : 9312428                                                              #
#  Turma: MAC5768 Visão e Processamento de Imagens                             #
#  Prof.: Ronaldo Fumio Hashimoto                                              #
################################################################################

## EP3 - Segmentação

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

import cv2

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)

#### Importa os metadados

In [2]:
metadata = pd.read_csv(rf"../metadados.csv", sep=';')

#### Criação da tabela sumária geral:

In [3]:

## Lista todas as imagens dos dados do EP2

path_list = []
names = []
path_folder = []
for path, subdirs, files in os.walk("../data_EP2/"):
    for name in files:
        path_list.append((os.path.join(path, name)))
        path_folder.append(path.split(rf'/')[-1])
        names.append(name)

In [4]:
# Cria um dataframe com todas as imagens, diretórios e pastas do Ep2
df_images = pd.DataFrame({'path' : path_list,
                          'img' : names,
                          'folder' : path_folder
                          })

In [5]:
# Define uma nova coluna "classe" para agrupar todas as classes presentes em cada imagem

metadata['classe'] = metadata['classe_1']
metadata['n_objetos'] = 1

for i in range(2, 6):
    metadata['classe'] += ', ' + metadata.apply(lambda row: row['classe_' + str(i)] if pd.notnull(row['classe_' + str(i)]) else '', axis=1)
    metadata['n_objetos'] += metadata.apply(lambda row: 1 if pd.notnull(row['classe_' + str(i)]) else 0, axis=1)


# Remove ',' no final do string
metadata['classe'] = metadata['classe'].str.rstrip(', ')

metadata['classe'] = metadata['classe'].str.rstrip(', ')

In [6]:
metadata.head(2)

,img,classe_1,classe_2,classe_3,classe_4,classe_5,fundo,iluminacao,classe,n_objetos
0,001.JPEG,garfo,pilha,faca,colher,NaN,sofa,interno-claro,"garfo, pilha, faca, colher",4
1,002.JPEG,som,livro,capacete,NaN,NaN,sofa,interno-claro,"som, livro, capacete",3


In [7]:
# Separa as palavras por vírgula e cria uma lista única de palavras
palavras_unicas = set(palavra.strip() for palavra in ','.join(metadata['classe']).split(',') if palavra)
# Converte o conjunto de volta para uma lista
lista_palavras_unicas = list(palavras_unicas)

##### TABELA SUMÁRIA

In [8]:

# Define type como o tipo de transformação utilizada
df_images['type'] = df_images['path'].str.split(rf'\\').str[-2]
df_images['type'] = df_images['type'].str.split(rf'/').str[-1]
# Garantindo que o nome tenha JPEG maiúsculo
df_images['img'] = df_images['img'].str.upper()

In [9]:
# Cria a relação de todas as imagens com a sua classe e iluminação, através do metadado:

metadata = df_images.merge(metadata, on='img', how='left')

In [19]:
metadata_unique = metadata.drop_duplicates('img')

#### Criação da Tabela Detalhada por Classe 

In [12]:
# pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org scikit-image

In [54]:
from skimage.measure import label, regionprops

# Lista para armazenar dados para o CSV

dados_csv = []

# Função para suavizar a imagem
def apply_smoothing(img, method='gaussian'):
    if method == 'gaussian':
        return cv2.GaussianBlur(img, (5, 5), 0)
    elif method == 'median':
        return cv2.medianBlur(img, 5)
    elif method == 'bilateral':
        return cv2.bilateralFilter(img, 9, 75, 75)
    else:
        raise ValueError('Unknown smoothing method.')

# Função para detecção de bordas gaussian, median, bilateral, gradient, canny, 'marr_hildreth'
def edge_detection(img, method='canny'):
    if method == 'gradient':
        grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        grad_mag = np.sqrt(grad_x**2 + grad_y**2)
        grad_mag = np.uint8(grad_mag / np.max(grad_mag) * 255)
        _, binary_img = cv2.threshold(grad_mag, 100, 255, cv2.THRESH_BINARY)
        return binary_img
    elif method == 'canny':
        return cv2.Canny(img, 50, 150)
    elif method == 'marr_hildreth':
        blurred = cv2.GaussianBlur(img, (0, 0), 1.0)
        log = cv2.Laplacian(blurred, cv2.CV_64F)
        log = np.uint8((log - log.min()) / (log.max() - log.min()) * 255)
        _, binary_img = cv2.threshold(log, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        return binary_img
    else:
        raise ValueError('Unknown edge detection method.')

# Função para traçar os contornos
def draw_contours(img, contours):
    img_contours = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(img_contours, contours, -1, (255, 0, 255), 1)
    return img_contours

# Função para filtrar contornos próximos das bordas
def filter_contours_near_edges(contours, img_shape, margin=10):
    height, width = img_shape
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x > margin and y > margin and (x + w) < (width - margin) and (y + h) < (height - margin):
            filtered_contours.append(contour)
    return filtered_contours


In [55]:
### Calcular os valores das áreas dos objetos

In [51]:

# Função principal para processar as imagens
def process_images(input_dir, smoothing_method='gaussian', edge_method='canny'):
    
    for root, dirs, files in os.walk(input_dir):
        print(files)
        for file in files:
            print(file)
            print(root)

            # if file in file.endswith('.jpeg') or file.endswith('.JPEG'):
            #file.endswith('.jpeg') or file.endswith('.JPEG'):
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)


            if img is not None:
                print(file.upper())

                # Aplicar a suavização
                img_smoothed = apply_smoothing(img, smoothing_method)
                n = int(metadata_unique.loc[metadata_unique['img'] == file.upper()]['n_objetos'].item()) #
                    
                    
                # Aplicar a detecção de bordas
                img_edges = edge_detection(img_smoothed, edge_method)

                # Encontrar contornos
                contours, hierarchy = cv2.findContours(img_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                # Filtrar contornos próximos das bordas
                contours = filter_contours_near_edges(contours, img.shape)

                label_img = label(img_smoothed, connectivity=2)
                props = regionprops(label_img)

                # Calcular a área dos contornos
                areas = []
                for c in contours:
                    areas.append(cv2.contourArea(c))


                areas_sorted = sorted(areas, reverse=True)

                major_contours = []
                print (areas_sorted)
                print(n)

                try:

                    for c in contours:
                        if len(areas_sorted) >= n:
                            if (cv2.contourArea(c)) > float(areas_sorted[n]):
                                major_contours.append(c)
                        else:
                            if (cv2.contourArea(c)) > len(areas_sorted):
                                major_contours.append(c)
                    img_contours = draw_contours(img, major_contours)
                    # Desenhar os contornos maiores na imagem original
    
                    



                    
    
                except:
                    pass

                        # # Filtrar props com base nas áreas dos maiores contornos
                props_filtradas = [prop for prop in props if prop.area in areas]
                props_img = props_filtradas

                # Ordenar as áreas do maior para o menor valor
                object_sorted = sorted(zip(areas, props_img), key=lambda x: x[0], reverse=True)
                
                
                # Traçar apenas os contornos das maiores áreas
                #n = len(areas_sorted) // 2  # Por exemplo, usar metade das áreas maiores
                # Para cada uma das três maiores áreas, recuperar diâmetro e menor eixo

                maiores_areas_props = areas_sorted[:n]
                maiores_obj_props = object_sorted[:n]


                for idx, (area, prop) in enumerate(maiores_obj_props):
                    if area is not None and prop is not None:
                        try:
                            diameter = prop.major_axis_length  # Supondo que você queira o diâmetro
                            minor_axis = prop.minor_axis_length  # Supondo que você queira o menor eixo
                            centroid = prop.centroid  # Obter o centroide do contorno
                            centroid_x, centroid_y = centroid[1], centroid[0]  # Coordenadas x e y do centroide
                        except ValueError as e:
                            print(f"Erro ao calcular propriedades: {e}")
                            diameter = None
                            minor_axis = None           
                            centroid_x = None
                            centroid_y = None
                    else:
                        area = ''
                        diameter = ''
                        minor_axis = ''
                        centroid_x = ''
                        centroid_y = ''
                
                    # Adicionar os dados para o CSV
                    dados_csv.append({
                        'img': file,  # Preencha conforme necessário  # Preencha conforme necessário
                        'path_origin': img_path,
                        'Objeto': '',  # Preencha conforme necessário
                        'Classe': '',  # Preencha conforme necessário
                        'n_contorno': idx,
                        'Área': f'{area:.10f}',
                        'Diâmetro': f'{diameter:.15f}',
                        'Minor': f'{minor_axis:.15f}',
                        'Centroide_X': f'{centroid_x:.4f}',
                        'Centroide_Y': f'{centroid_y:.4f}'
                    })

                        # # Imprimir os dados
                        # print(f"Imagem: {imagem_path} - Área: {area}, Diâmetro: {diameter}, Menor: {minor_axis}")
    


                    # major_contours = [c for c in maiores_areas_props if c and c is not None]
                    # print(major_contours)


                    # except:
                    #     major_contours = [c for c in maiores_areas_props]
                    # Desenhar os contornos maiores
    return dados_csv


# Caminhos para os diretórios das imagens suavizadas
# caminho_suavizado_augmented = os.path.join('augmentedDataset')
# caminho_suavizado_normalized = os.path.join('normalizedDataset')
# output_dir_augmented = os.path.join('augmentedDataset_segmented')
# output_dir_normalized = os.path.join('normalizedDataset_segmented')


caminho_selected_imgs_original = rf'..\data_EP3\selectedData\origin'


# # Processar e segmentar as imagens nos datasets suavizados
# process_images(caminho_suavizado_augmented, output_dir_augmented, 'gaussian', 'canny')
# process_images(caminho_suavizado_normalized, output_dir_normalized, 'gaussian', 'canny')


data_characteristics = process_images(caminho_selected_imgs_original, 'gaussian', 'canny')

['021.JPEG', '052.JPEG']
021.JPEG
..\data_EP3\selectedData\origin\augmentedDataset\laplaciano
021.JPEG
[68900.5, 66508.5, 1102.5, 199.5, 133.0, 119.5, 104.0, 73.0, 68.5, 52.5, 52.0, 50.0, 48.5, 44.0, 40.5, 39.5, 39.5, 39.0, 36.5, 36.0, 36.0, 35.0, 33.5, 30.5, 26.5, 26.5, 25.5, 24.5, 24.0, 23.0, 22.0, 21.5, 21.0, 20.5, 16.5, 16.0, 16.0, 15.5, 15.0, 15.0, 14.0, 13.0, 12.5, 11.5, 10.0, 9.5, 9.5, 9.5, 9.5, 8.5, 8.5, 8.0, 8.0, 7.5, 7.5, 7.5, 7.5, 7.0, 7.0, 7.0, 6.5, 5.5, 5.5, 5.5, 5.5, 5.5, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.5]
4
052.JPEG
..\data_EP3\selectedData\origin\augmentedDataset\laplaciano
052.JPEG
[684.5, 329.0, 148.5, 88.5, 76.5, 68.0, 68.0, 57.0, 18.5, 17.5, 15.0, 14.5, 14.5, 14.0, 13.5, 12.5, 12.5, 11.5, 11.5, 9.5, 9.5, 8.5, 7.0, 7.0]
3


In [52]:
df_characteristics = pd.DataFrame(data_characteristics)

In [53]:
df_characteristics.head()

,img,path_origin,Objeto,Classe,n_contorno,Área,Diâmetro,Minor,Centroide_X,Centroide_Y
0,021.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\laplaciano\021.JPEG,,,0,68900.5000000000,4.318667337728677,3.679464844031199,287.6154,6.6154
1,021.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\laplaciano\021.JPEG,,,1,66508.5000000000,2.000000000000000,2.000000000000000,447.5000,5.5000
2,021.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\laplaciano\021.JPEG,,,2,1102.5000000000,19.489045764668695,2.814604741456738,487.4615,22.4103
3,021.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\laplaciano\021.JPEG,,,3,199.5000000000,8.000000000000000,2.000000000000000,511.5000,3.0000
4,052.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\laplaciano\052.JPEG,,,0,684.5000000000,13.091133555875818,2.065591117977291,750.6667,4.3333


In [30]:
df_characteristics.to_csv('..\image_characteristics.csv', index=False)